# EDA Notworking2Networking attendance

Import libraries

In [1]:
import gspread
from oauth2client.service_account import ServiceAccountCredentials

from df2gspread import df2gspread as d2g
from gspread_dataframe import get_as_dataframe, set_with_dataframe

In [2]:
import numpy as np
import pandas as pd
from datetime import datetime

import plotly.express as px
import plotly.graph_objects as go
import matplotlib.pyplot as plt
import seaborn as sns

import os
import json
from decouple import config

## Read google sheets

Define the scope

In [3]:
# define the scope and credentials
SCOPE = ["https://spreadsheets.google.com/feeds", "https://www.googleapis.com/auth/drive"]
CREDENTIALS = json.loads(config('CRED_GCP'))


Credentials

In [4]:
# add credentials to the account
#credentials = ServiceAccountCredentials.from_json_keyfile_name(API_FILE, SCOPE)
credentials = ServiceAccountCredentials.from_json_keyfile_dict(CREDENTIALS, SCOPE)

# authorize the clientsheet
client = gspread.authorize(credentials)

# get the instance of the Spreadsheet
sheet = client.open('Copy of N2N - Database')

# get the first sheet of the Spreadsheet
sheet_instance = sheet.get_worksheet(0)

In [5]:
# get the total number of columns
print(sheet_instance.col_count)
print(sheet_instance.row_count)

16
5717


Database

In [6]:
data = sheet_instance.get_all_values()
df = pd.DataFrame(data[1:], columns=data[0])

In [7]:
df.head()

,#,Date,City,Season,Industry / Event,Format,Attendance,Email,First Name,Last Name,Country of Origin,Area of Expertise,Employment Status,Employer,Dream Job,Linkedin
0,1,"January 14, 2021",Toronto,1,HR,Online,Checked In,fabysantanar@gmail.com,Fabiana,Santana,Venezuela,"Employee Experience, Business Partner",Employed,Spin Master,,
1,1,"January 14, 2021",Toronto,1,HR,Online,Checked In,davhd21@gmail.com,David,Herrera,Mexico,Talent,Employed,Korn Ferry,,
2,1,"January 14, 2021",Toronto,1,HR,Online,Checked In,karla.cadario@wilsonhcg.com,Karla,Cadario Moreno,Bolivia,Reclutamiento y Seleccion,Employed,WilsonHCG,,
3,1,"January 14, 2021",Toronto,1,HR,Online,Checked In,eri_mabu@hotmail.com,Erika,Martínez,Colombia,Recruiting,Employed,Accenture,,
4,1,"January 14, 2021",Toronto,1,HR,Online,Checked In,padronalfredo@outlook.com,Alfredo,Padron,Venezuela,Tech,Employed,Scotiabank,,


### Descriptive statistics

In [8]:
df.describe()

,#,Date,City,Season,Industry / Event,Format,Attendance,Email,First Name,Last Name,Country of Origin,Area of Expertise,Employment Status,Employer,Dream Job,Linkedin
count,5716,5716,5716,5716,5716,5716,5716,5716,5716,5716,5716,5716,5716,5716,5716,5716
unique,192,176,2,10,44,2,3,3006,1286,1875,26,2549,6,1845,2416,2412
top,115,"July 17, 2023",Toronto,9,IT,Online,Attending,andresbcrra@gmail.com,Juan,Garcia,Colombia,,Unemployed and looking for opportunities,,,
freq,310,310,5607,2834,541,3336,3532,23,113,61,2064,1043,2130,2030,2114,2442


In [9]:
total_registered = df['Attendance'].count()
total_registered

5716

In [10]:
total_attendance = df[df['Attendance'] == 'Checked In']['Attendance'].count()
total_attendance

2091

In [11]:
df['Season'] = pd.to_numeric(df['Season'], errors='coerce')

In [12]:
toronto_season = df[df['City']=='Toronto']['Season'].max()
toronto_season

10

In [13]:
montreal_season = df[df['City']=='Montreal']['Season'].max()
montreal_season

1

In [14]:
df[df['City']=='Montreal'][['Date','Season']]

,Date,Season
3402,"September 20, 2023",1
3403,"September 20, 2023",1
3404,"September 20, 2023",1
3405,"September 20, 2023",1
3406,"September 20, 2023",1
...,...,...
4742,"November 9, 2023",1
4743,"November 9, 2023",1
4744,"November 9, 2023",1
4745,"November 9, 2023",1


In [15]:
df.loc[df['City']=='Montreal',['Date','Season']]

,Date,Season
3402,"September 20, 2023",1
3403,"September 20, 2023",1
3404,"September 20, 2023",1
3405,"September 20, 2023",1
3406,"September 20, 2023",1
...,...,...
4742,"November 9, 2023",1
4743,"November 9, 2023",1
4744,"November 9, 2023",1
4745,"November 9, 2023",1


### Employment Status Attendees

In [16]:
filtered_df = df[(df['Employment Status']!='') &
                 (df['Employment Status']!='Maternity Leave / Full-time Mom') &
                 (df['Employment Status']!='Entrepreneur')
                 ]
employment_status = filtered_df['Employment Status'].value_counts()
employment_status


Employment Status
Unemployed and looking for opportunities    2130
Employed and looking for opportunities      1613
Employed                                     964
Name: count, dtype: int64

In [17]:
fig = px.bar(x=employment_status.index, y=employment_status.values, labels={'x': 'Employment Status', 'y': 'Count'})
fig.show()

### Attendee's Countries

In [18]:
filtered_df = df[df['Country of Origin']!='']
top_6_countries = filtered_df['Country of Origin'].value_counts().nlargest(6)
top_6_countries

Country of Origin
Colombia              2064
Mexico                 566
Venezuela              392
Peru                   385
Dominican Republic     241
Ecuador                187
Name: count, dtype: int64

In [19]:
figure = px.bar(x=top_6_countries.values, y=top_6_countries.index, labels={'x': '', 'y': 'Employment Status'})
figure.show()

In [20]:
top_6_countries_sorted = top_6_countries.sort_values(ascending=True)

figure = px.bar(x=top_6_countries_sorted.values, y=top_6_countries_sorted.index, labels={'x': '', 'y': 'Employment Status'})
figure.show()

In [21]:
df['Country of Origin']

0       Venezuela
1          Mexico
2         Bolivia
3        Colombia
4       Venezuela
          ...    
5711             
5712             
5713             
5714             
5715             
Name: Country of Origin, Length: 5716, dtype: object

In [22]:
country_counts = df['Country of Origin'].value_counts().reset_index()
country_counts.columns = ['country', 'attendants']

In [23]:
country_counts

,country,attendants
0,Colombia,2064
1,,1164
2,Mexico,566
3,Venezuela,392
4,Peru,385
5,Dominican Republic,241
6,Ecuador,187
7,Chile,161
8,Canada,106
9,Honduras,86


In [24]:


fig5 = px.choropleth(
    country_counts,
    locations='country',
    locationmode='country names',
    color='attendants',
    hover_name='country',
    color_continuous_scale='Viridis_r',
    #range_color=[0,3000],
    title='NotWorking2Networking',
)

fig5.update_geos(
    #showcountries =True,
    showcoastlines=True,
    coastlinecolor='RebeccaPurple',
    showland=True,
    landcolor='LightGrey',
    showocean=True,
    oceancolor='LightBlue',
    fitbounds= 'locations',
    projection_type='orthographic',
)

fig5.update_layout(
    geo=dict(showframe=False, showcoastlines=False),
    coloraxis_colorbar=dict(title='Attendants by country'),
)


fig5.show()

In [25]:
'''
world = gpd.read_file(gpd.datasets.get_path('naturalearth_lowres'))

# Count the number of occurrences of each country in the DataFrame
country_counts = df['Country of Origin'].value_counts().reset_index()
country_counts.columns = ['name', 'count']

# Merge the world shapefile with the country counts DataFrame
world = world.merge(country_counts, how='left', left_on='name', right_on='name')

# Plotting the map
fig, ax = plt.subplots(1, 1, figsize=(15, 10))
world.boundary.plot(ax=ax)
world.plot(column='count', ax=ax, legend=True, legend_kwds={'label': "Number of Countries"})
plt.show()
'''

'\nworld = gpd.read_file(gpd.datasets.get_path(\'naturalearth_lowres\'))\n\n# Count the number of occurrences of each country in the DataFrame\ncountry_counts = df[\'Country of Origin\'].value_counts().reset_index()\ncountry_counts.columns = [\'name\', \'count\']\n\n# Merge the world shapefile with the country counts DataFrame\nworld = world.merge(country_counts, how=\'left\', left_on=\'name\', right_on=\'name\')\n\n# Plotting the map\nfig, ax = plt.subplots(1, 1, figsize=(15, 10))\nworld.boundary.plot(ax=ax)\nworld.plot(column=\'count\', ax=ax, legend=True, legend_kwds={\'label\': "Number of Countries"})\nplt.show()\n'

### Pie charts

In [26]:
df.columns

Index(['#', 'Date', 'City', 'Season', 'Industry / Event', 'Format',
       'Attendance', 'Email', 'First Name', 'Last Name', 'Country of Origin',
       'Area of Expertise', 'Employment Status', 'Employer', 'Dream Job',
       'Linkedin'],
      dtype='object')

In [27]:
figure = px.pie(df, names='Format', title='Attendees by Event mode')
figure.show()

In [28]:
figure = px.pie(df, names='City', title='Attendees by City')
figure.show()

### Notworking to Networking Over Time

In [29]:
df.columns

Index(['#', 'Date', 'City', 'Season', 'Industry / Event', 'Format',
       'Attendance', 'Email', 'First Name', 'Last Name', 'Country of Origin',
       'Area of Expertise', 'Employment Status', 'Employer', 'Dream Job',
       'Linkedin'],
      dtype='object')

In [30]:
df['Date'] = pd.to_datetime(df['Date'])

In [31]:
attendance_grouped = df.groupby(['Date', 'Attendance']).size().reset_index(name='count')
attendance_grouped.loc[attendance_grouped['Attendance'] == 'Attending', 'Attendance'] = 'Not Attending'


In [32]:
attendance_grouped['cumulative_count'] = attendance_grouped.groupby('Attendance')['count'].cumsum()

In [33]:
attendance_grouped['total_count'] = attendance_grouped['count'].cumsum()


In [34]:
attendance_grouped

,Date,Attendance,count,cumulative_count,total_count
0,2021-01-14,Checked In,10,10,10
1,2021-01-21,Checked In,17,27,27
2,2021-01-30,Checked In,22,49,49
3,2021-02-04,Checked In,22,71,71
4,2021-02-09,Checked In,25,96,96
...,...,...,...,...,...
297,2024-02-07,Not Attending,29,3541,5598
298,2024-02-07,Checked In,34,2091,5632
299,2024-02-07,Not Attending,6,3547,5638
300,2024-02-08,Not Attending,67,3614,5705


In [35]:
# Creating a line plot using Plotly Express
fig = px.line(attendance_grouped, x='Date', y='count', color='Attendance', markers=True, title='Attendance Count Over Time')
fig.show()


In [36]:
fig = px.line(attendance_grouped, x='Date', y= 'total_count', markers=True, title='Attendance Count Over Time')
fig.show()

In [37]:
fig = px.line(attendance_grouped, x='Date', y=['count', 'total_count'], color='Attendance', markers=True, title='Attendance Count Over Time')
fig.show()

### Attendees by Industry

In [38]:
df[['#', 'Industry / Event', 'Format', 'Attendance']]

,#,Industry / Event,Format,Attendance
0,1,HR,Online,Checked In
1,1,HR,Online,Checked In
2,1,HR,Online,Checked In
3,1,HR,Online,Checked In
4,1,HR,Online,Checked In
...,...,...,...,...
5711,192,Project Management,In Person,Not Attending
5712,192,Project Management,In Person,Not Attending
5713,192,Project Management,In Person,Not Attending
5714,192,Project Management,In Person,Not Attending


In [39]:
### Check this

In [40]:
filtered_df = df[(df['Industry / Event']!='Workshop: LinkedIn Workshop to Advance Your Career') &
                 (df['Industry / Event']!='Workshop: Insider Secrets to Landing Ideal Jobs (for Newcomers)') &
                 (df['Industry / Event']!='Workshop: Secrets to Crafting The Perfect Job Application by Izzy Piyale-Sheard') &
                 (df['Industry / Event']!='Workshop: Top 22 Tips to Get a Job in 2022') &
                 (df['Industry / Event']!='Workshop: How to Write Business English (for Newcomers)') &
                 (df['Industry / Event']!='End of Season Online Meetup') &
                 (df['Industry / Event']!='S8 Party') &
                 (df['Industry / Event']!='End of Season In Person Meetup') &
                 (df['Industry / Event']!='S7 Party') &
                 (df['Industry / Event']!='Volunteer Training')]

#top_9_industry = filtered_df[['Industry / Event','Format']].value_counts().nlargest(20).to_frame().reset_index()
top_9_industry = filtered_df[['Industry / Event','Format']].value_counts().to_frame().reset_index()

In [41]:
top_9_industry

,Industry / Event,Format,count
0,IT,Online,293
1,IT,In Person,248
2,Project Management,Online,220
3,Marketing,Online,218
4,HR,Online,213
...,...,...,...
61,Moms,In Person,8
62,Mental Health,In Person,8
63,Medicine & Pharmacy,Online,7
64,"Crypto, NFT & Blockchain",Online,7


In [42]:
fig = px.bar(top_9_industry,x='Industry / Event', y='count',color='Format', barmode='group',labels={'x': 'Industry / Event', 'y': 'Format'})
fig.show()


#### Average attendees by Industry

In [43]:
df_attendees = df[['#','Industry / Event', 'Format', 'Attendance']]
#df_attendees = df[(df['Attendance'] == 'Checked In') | (df['Attendance'] == 'Attending')]
#df_attendees = df[df['Attendance'] == 'Not Attending']

In [44]:
df_attendees = df_attendees[['#', 'Industry / Event','Format']].value_counts().sort_index().reset_index(name='Attendants')
df_attendees

,#,Industry / Event,Format,Attendants
0,1,HR,Online,10
1,10,Hospitality & Tourism,Online,20
2,100,Social Work & Mental Health,In Person,34
3,101,Retail,Online,14
4,102,Retail,In Person,15
...,...,...,...,...
188,95,S7 Party,In Person,61
189,96,Workshop: Secrets to Crafting The Perfect Job ...,Online,36
190,97,Engineering,Online,33
191,98,Engineering,In Person,35


In [45]:
#df_attendees['Attendance'].value_counts()

In [46]:
#df_attendees = df_attendees[['Industry / Event', 'Format', 'Attendants']]

In [47]:
df_attendees[df_attendees['Industry / Event'] == 'Workshop: LinkedIn Workshop to Advance Your Career']

,#,Industry / Event,Format,Attendants
18,115,Workshop: LinkedIn Workshop to Advance Your Ca...,Online,310
21,118,Workshop: LinkedIn Workshop to Advance Your Ca...,Online,231


In [48]:
#avg_attendance = df_attendees.groupby(['Industry / Event', 'Format'])['Attendants'].mean().reset_index()
#avg_attendance = df_attendees.groupby(['Industry / Event'])['Attendants'].mean().reset_index()
#avg_attendance

In [49]:
# Grouping by "Industry / Event" and "Format" and calculating the mean, sum, and count of "attendants"
#attendance_stats = df_attendees.groupby(['Industry / Event', 'Format'])['Attendants'].agg(['mean', 'sum', 'count']).reset_index()

# Rename the columns for clarity
#attendance_stats.columns = ['Industry / Event', 'Format', 'Average Attendance', 'Total Attendance', 'Number of Events']

In [50]:
#attendance_stats

In [51]:
#avg_attendance[avg_attendance['Industry / Event'] == 'Workshop: LinkedIn Workshop to Advance Your Career']

In [52]:
#avg_attendance[avg_attendance['Industry / Event'] == 'IT']

In [53]:
df_attendees[df_attendees['Industry / Event']=='HR']

,#,Industry / Event,Format,Attendants
0,1,HR,Online,10
16,113,HR,Online,40
17,114,HR,In Person,42
31,127,HR,Online,53
33,129,HR,In Person,52
99,189,HR,Online,46
100,19,HR,Online,9
102,191,HR,In Person,69
105,20,HR,In Person,1
150,61,HR,Online,25


In [54]:
#attendance_grouped = df.groupby(['Date', 'Attendance']).size().reset_index(name='count')
#attendance_grouped.loc[attendance_grouped['Attendance'] == 'Attending', 'Attendance'] = 'Not Attending'

In [55]:
#attendance_grouped

In [56]:
#attendance_grouped[attendance_grouped['Industry / Event']=='HR']

In [57]:
df_attendees

,#,Industry / Event,Format,Attendants
0,1,HR,Online,10
1,10,Hospitality & Tourism,Online,20
2,100,Social Work & Mental Health,In Person,34
3,101,Retail,Online,14
4,102,Retail,In Person,15
...,...,...,...,...
188,95,S7 Party,In Person,61
189,96,Workshop: Secrets to Crafting The Perfect Job ...,Online,36
190,97,Engineering,Online,33
191,98,Engineering,In Person,35


In [58]:
#df_attendees_filtered = df_attendees[df_attendees['Attendance'] != 'Attending']
#df_attendees_filtered

In [59]:
#df_attendees=df_attendees.groupby(['Industry / Event', 'Format'])['Count'].sum().reset_index()

In [60]:
df_average_attendees = df_attendees.groupby(['Industry / Event', 'Format']).agg({'Attendants': ['sum', 'count']}).reset_index()
df_average_attendees.columns = ['Industry / Event', 'Format', 'Sum_Count', 'Attendants']

In [61]:
df_average_attendees['Average']= df_average_attendees['Sum_Count']/df_average_attendees['Attendants']

In [62]:
df_average_attendees

,Industry / Event,Format,Sum_Count,Attendants,Average
0,Accounting,In Person,10,1,10.000000
1,Accounting,Online,22,1,22.000000
2,"Advertising, Media & Film",In Person,94,3,31.333333
3,"Advertising, Media & Film",Online,88,3,29.333333
4,Architecture,In Person,38,2,19.000000
...,...,...,...,...,...
71,Workshop: How to Write Business English (for N...,Online,116,1,116.000000
72,Workshop: Insider Secrets to Landing Ideal Job...,Online,184,1,184.000000
73,Workshop: LinkedIn Workshop to Advance Your Ca...,Online,541,2,270.500000
74,Workshop: Secrets to Crafting The Perfect Job ...,Online,36,1,36.000000


In [63]:
df_average_attendees[df_average_attendees['Industry / Event']=='IT']

,Industry / Event,Format,Sum_Count,Attendants,Average
44,IT,In Person,248,9,27.555556
45,IT,Online,293,9,32.555556


In [64]:
df.columns

Index(['#', 'Date', 'City', 'Season', 'Industry / Event', 'Format',
       'Attendance', 'Email', 'First Name', 'Last Name', 'Country of Origin',
       'Area of Expertise', 'Employment Status', 'Employer', 'Dream Job',
       'Linkedin'],
      dtype='object')

In [65]:
#filtered_df['Industry / Event'].unique()

In [66]:
filtered_df2 = df_average_attendees[(df_average_attendees['Industry / Event']!='Workshop: LinkedIn Workshop to Advance Your Career') &
                                    (df_average_attendees['Industry / Event']!='Workshop: Insider Secrets to Landing Ideal Jobs (for Newcomers)') &
                                    (df_average_attendees['Industry / Event']!='Workshop: Secrets to Crafting The Perfect Job Application by Izzy Piyale-Sheard') &
                                    (df_average_attendees['Industry / Event']!='Workshop: Top 22 Tips to Get a Job in 2022') &
                                    (df_average_attendees['Industry / Event']!='Workshop: How to Write Business English (for Newcomers)') &
                                    (df_average_attendees['Industry / Event']!='End of Season Online Meetup') &
                                    (df_average_attendees['Industry / Event']!='S8 Party') &
                                    (df_average_attendees['Industry / Event']!='End of Season In Person Meetup') &
                                    (df_average_attendees['Industry / Event']!='S7 Party') &
                                    (df_average_attendees['Industry / Event']!='Volunteer Training')]


In [67]:
filtered_df2[filtered_df2['Industry / Event']=='IT']

,Industry / Event,Format,Sum_Count,Attendants,Average
44,IT,In Person,248,9,27.555556
45,IT,Online,293,9,32.555556


In [68]:
filtered_df2

,Industry / Event,Format,Sum_Count,Attendants,Average
0,Accounting,In Person,10,1,10.000000
1,Accounting,Online,22,1,22.000000
2,"Advertising, Media & Film",In Person,94,3,31.333333
3,"Advertising, Media & Film",Online,88,3,29.333333
4,Architecture,In Person,38,2,19.000000
...,...,...,...,...,...
63,Retail,Online,29,3,9.666667
66,Sales,In Person,23,1,23.000000
67,Sales,Online,38,2,19.000000
68,Social Work & Mental Health,In Person,62,2,31.000000


In [69]:
#top_average = filtered_df2[['Industry / Event','Format']].value_counts().nlargest(20).to_frame().reset_index()
top_average = filtered_df2[['Industry / Event','Format','Average']].value_counts().to_frame().reset_index()

In [70]:
top_average =top_average.sort_values('Average',ascending=False)

In [71]:
top_average 

,Industry / Event,Format,Average,count
42,"Banking, Finance & Accounting",In Person,74.0,1
43,"Banking, Finance & Accounting",Online,69.0,1
53,Data Analysis,Online,59.0,1
52,Data Analysis,In Person,57.0,1
33,Executive & Adm Assistants,Online,44.0,1
...,...,...,...,...
2,Fashion,Online,7.5,1
15,Medicine & Pharmacy,Online,7.0,1
51,"Crypto, NFT & Blockchain",Online,7.0,1
32,Fashion,In Person,6.0,1


In [72]:
fig = px.histogram(top_average,x='Industry / Event', y = 'Average',color='Format',  barmode='group')
fig.show()